## Example: Three-State Markov Model
In this example, let's construct and then sample a three-state Markov model, with three states $\mathcal{S}\equiv\left\{1,2,3\right\}$, shown below:
<div>
    <center>
        <img src="figs/Fig-ThreeState-MM-Schematic.svg" width="580"/>
    </center>
</div>
where the $p_{ij}$ values describe the probability of moving from state $s_{i}$ to state $s_{j}$ in the next time step. The states in the model could represent many things, for example an investors attitide toward an asset.

## Setup
Let's load some required packages for the example by calling the `include(...)` function on our [initialization file `Include.jl`](Include.jl):

In [3]:
include("Include.jl")

    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
   Resolving package versions...
    Updating `~/Desktop/julia_work/CHEME-5660-CourseRepository-Fall-2024/lectures/week-13/L13a/Project.toml`
  [336ed68f] + CSV v0.10.15
  [5ae59095] + Colors v0.13.0
  [a93c6f00] + DataFrames v1.7.0
  [b4f34e82] + Distances v0.10.12
  [31c24e10] + Distributions v0.25.113
  [5789e2e9] + FileIO v1.16.5
  [09f84164] + HypothesisTests v0.11.3
  [033835bb] + JLD2 v0.5.8
  [91a5bcdd] + Plots v1.40.9
  [10745b16] + Statistics v1.11.1
  [2913bbd2] + StatsBase v0.34.3
  [f3b207a7] + StatsPlots v0.15.7
  [4119e0bf] + VLQuantitativeFinancePackage v1.0.0-DEV `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git#main`
  [37e2e46d] ~ LinearAlgebra ⇒ v1.11.0
    Updating `~/Desktop/julia_work/CHEME-5660-CourseRepository-Fall-2024/lectures/week-13/L13a/Manifest.toml`
  [621f4979] + AbstractFFTs v1.5.0
  [7d9f7c33] + Accessors v0.1.38
  [79e6a3ab] + Adapt v4.1.1
  [66da

encode

## Prerequisites 
Before we start this example, let's set up the `iterate(...)` method and specify some constants. We'll use the `iterate(...)` method to compute the stationary distribution $\pi$.
```julia
iterate(P::Array{Float64,2}; 
        maxcount::Int = 100, ϵ::Float64 = 0.1) -> Array{Float64,2}
```
> Iteratively computes a stationary distribution. Computation stops if ||P_new - P|| < ϵ or the max number of iterations is hit. 

`Unhide` the code block below to see the implementation of the `iterate(...)` method:

In [6]:
function iterate(P::Array{Float64,2}; 
        maxcount::Int = 100, ϵ::Float64 = 0.1)::Array{Float64,2}

    # initialize -
    counter = 1; # initialize the iteration counter
    is_ok_to_stop = false; # flag for while loop
    P_new = nothing; # initialize P_new matrix
    
    # main loop - iterate until the difference ||P_new - P|| <= ϵ -or- we run out of iterations
    while (is_ok_to_stop == false)
        P_new = P^(counter+1); # compute new P matrix by raising to counter + 1 power
        if (norm(P_new - P) <= ϵ || counter >= maxcount)
            is_ok_to_stop = true;
        end
        counter += 1; # update the counter
    end

    # return -
    return P_new;
end;

#### Constants
In the simulations below, we'll need some constant values that we set here. In particular, we set a value for the `number_of_hidden_states` variable, the `number_of_simulation_steps` variable (the number of steps that we take in a Markov Chain), and the `number_of_samples ` variable:

In [8]:
number_of_hidden_states = 3; # how many states do we have?
number_of_simulation_steps = 10000; # number of simulation steps
number_of_samples = 10000; # number of samples

## Task 1: Setup transition matrix for a three-state Markov Model
In this task, we'll set up the transition matrix $\mathbf{P}$ for the three-state [Markov chain model](https://en.wikipedia.org/wiki/Markov_chain) shown above.
* In this example we have three states $\mathcal{S}=\left\{1,2,3\right\}$ where the probability of moving between state $s_{i}\rightarrow{s_{j}}$ in the next time step is denoted as $p_{ij}\in\mathcal{P}$. The transition matrix is a $|\mathcal{S}|\times|\mathcal{S}|$ matrix with non-negative elements.

In [10]:
P = [
    0.05 0.95 0.0 ; # possible next state from state 1 
    0.6 0.2 0.2 ; # possible next state from state 2
    0.0 0.3 0.7 ; # possible next state from state 3
];

### Check: do the rows of the transition matrix $\mathbf{P}$ sum to `1`?
We know that the rows of the transition matrix $\mathbf{P}$ must sum to `1`, i.e., if we are in state $s_{i}\in\mathcal{S}$ at time $t$, then at time $t+1$ we have to be in $s_{j}\in\mathcal{S}$. 
* Let's check if the transition matrix $\mathbf{P}$ meets this criteria using the [@assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) by iterating over the rows of the transition matrix $\mathbf{P}$ and checking the sum of each row. If any row does not meet this criterion, an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) will be thrown.

In [12]:
for i ∈ 1:number_of_hidden_states
    @assert sum(P[i,:]) == 1
end

## Task 2: Compute the stationary distribution $\bar{\pi}$
In this task, we'll compute the stationary distribution $\pi$ for our three-state example [Markov chain](https://en.wikipedia.org/wiki/Markov_chain) shown above.

We'll compute the stationary distribution $\bar{\pi}$ using the iterative `iterate(...)` method. During each iteration, we compute the matrix power of transition matrix $\mathbf{P}$. We continue to iterate until we hit one of two possible conditions:
* The  `counter == maxcount,` at this point, the iteration stops, and the matrix $\mathbf{P}^{\star}$ is returned
* The iteration stops because difference between subsequent powers of the matrix $\mathbf{P}$ is smaller than some specified threshold $\epsilon$

In [14]:
π̄ = iterate(P, ϵ = 0.0000001) # this gives the stationary distribution

3×3 Matrix{Float64}:
 0.274809  0.435115  0.290076
 0.274809  0.435115  0.290076
 0.274809  0.435115  0.290076

### Check: Is the rank condition on the stationary distribution $\bar{\pi}$ correct?
Once we reach the stationary distribution, the rank of the stationary distribution $\bar{\pi}$ should be equal to `1`. Let's check whether this condition is true using the [@assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert). 
* If we do not meet this criterion, an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) will be thrown, and we should try to use more iterations or a tighter numerical tolerance value for $\epsilon$. We'll compute the rank using the [rank function](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.rank) which is exported by the [Julia Statistics package](https://docs.julialang.org/en/v1/stdlib/Statistics/#Statistics)

In [16]:
@assert rank(π̄) == 1

### Deeper: Compute Stationary Distribution $\bar{\pi}$ using the Left Matrix Vector Product
We can also approach the computation of the stationary distribution $\bar{\pi}$ by directly iterating the expression:
$$
\begin{equation*}
\pi_{n+1} = \pi_{1}\cdot\mathbf{P}^{n}\quad\,n=1,2,\dots
\end{equation*}
$$
where $\pi_{1}$ is our initial belief of the state distribution, and $\mathbf{P}$ is the transition matrix. As $n\rightarrow\infty$, i.e., as we do more iterations, the difference between subsequent iterations should become small $||\pi_{n+1}-\pi_{n}||<\epsilon$ for a non-periodic Markov chain, where $\pi_{n}\rightarrow\bar{\pi}$ as $n\rightarrow\infty$. 

In [18]:
π₁ = [0.0,1.0,0.0]; # initial state = state 2
direct_state_distribution = Dict{Int,Array{Float64,1}}();
direct_state_distribution[1] = π₁;
for n = 2:number_of_simulation_steps
    πᵢ = transpose(π₁)*(P)^(n-1)
    direct_state_distribution[n] =  transpose(πᵢ)
end
foreach(i-> println(direct_state_distribution[i]), 1:20);

[0.0, 1.0, 0.0]
[0.6, 0.2, 0.2]
[0.15, 0.6699999999999999, 0.18]
[0.4094999999999999, 0.33049999999999996, 0.26]
[0.21877499999999994, 0.533125, 0.2481]
[0.3308137499999999, 0.38889124999999997, 0.28029499999999996]
[0.2498754374999999, 0.4761398124999999, 0.27398475]
[0.2981776593749999, 0.4148050531249999, 0.28701728749999994]
[0.26379191484374986, 0.45233497328124994, 0.283873111875]
[0.2845905797109374, 0.4262312473203125, 0.2891781729687499]
[0.2699682773777342, 0.442360752080078, 0.28767097054218743]
[0.2789148651169334, 0.4312433050875194, 0.2898418297955468]
[0.27269172630835825, 0.43817033181725473, 0.2891379418743867]
[0.2765367854057707, 0.4334325889187073, 0.29003062567552157]
[0.2738863926215128, 0.43640565162188016, 0.28970795575660657]
[0.2755377106042036, 0.4343855900417952, 0.2900766993540006]
[0.27440823955528726, 0.4356609528885528, 0.2899308075561595]
[0.275116983710896, 0.43479926042208134, 0.2900837558670222]
[0.2746354054387935, 0.43534611336987405, 0.29001848119

## Task 3: Sample from our Markov Model using a Categorical Distribution
In this task, we compute a sequence of states for the three-state [Markov chain model](https://en.wikipedia.org/wiki/Markov_chain) and compare the frequency of those states to what we see for the stationary distribution $\bar{\pi}$.

### Sampling
We can get the dynamics, i.e., the sequences of states $s_{1},s_{2},\dotsc$ predicted by the [Markov model](https://en.wikipedia.org/wiki/Markov_model) by sampling the transition probability matrix $\mathbf{P}$ directly. We can do this using a [categorical distribution](https://en.wikipedia.org/wiki/Categorical_distribution), which models transition from state $i\rightarrow{j}$ in the next time step.

In [20]:
state_probability_dictionary = Dict{Int,Categorical}();
for i ∈ 1:number_of_hidden_states
    state_probability_dictionary[i] = Categorical(P[i,:])
end
state_probability_dictionary

Dict{Int64, Categorical{P} where P<:Real} with 3 entries:
  2 => Categorical{Float64, Vector{Float64}}(support=Base.OneTo(3), p=[0.6, 0.2…
  3 => Categorical{Float64, Vector{Float64}}(support=Base.OneTo(3), p=[0.0, 0.3…
  1 => Categorical{Float64, Vector{Float64}}(support=Base.OneTo(3), p=[0.05, 0.…

Let's generate `number_of_simulation_steps` worth of dynamic data by sampling the `state_probability_dictionary.` We store these simulation results in the `simulation_dictionary` dictionary, where the `key` holds the time index and the `value` is the system's state, i.e., $s_{i}\in\mathcal{S}$.
* We start by specifying an initial state `sᵢ = 1`. At each iteration of the loop, we pull out the [categorical distribution](https://en.wikipedia.org/wiki/Categorical_distribution) corresponding to the state $s_{i}$, i.e., the row in the transition matrix $\mathbf{P}$ corresponding to state $s_{i}$. We then generate the state at the next step by drawing a sample from the categorical distribution using the `rand(...)` function.

In [22]:
simulation_dictionary = Dict{Int,Int}();
sᵢ = 1; # harcode the start state: we also could draw from the stationary distribution
simulation_dictionary[1] = sᵢ;
for i ∈ 2:number_of_simulation_steps
    
    sᵢ = state_probability_dictionary[sᵢ] |> d -> rand(d); # get dcat for sᵢ, draw a sample -> next state
    simulation_dictionary[i] = sᵢ;  # capture the next state at t = i
    
end
foreach(i -> println("Soln: (t=$(i),s=$(simulation_dictionary[i]))"), 1:10) # look at first 10 steps

Soln: (t=1,s=1)
Soln: (t=2,s=2)
Soln: (t=3,s=1)
Soln: (t=4,s=2)
Soln: (t=5,s=3)
Soln: (t=6,s=2)
Soln: (t=7,s=1)
Soln: (t=8,s=2)
Soln: (t=9,s=3)
Soln: (t=10,s=3)


### Check: Do we recover the stationary distribution $\bar{\pi}$?
Like any stable dynamic system, e.g., concentration balances in a steady-state reactor, if we wait long enough, the system should approach a steady state (assuming the steady state exists and is stable). By analogy, if we take enough time steps for our random dynamic system, the distribution of states should follow the stationary distribution $\bar{\pi}$.

In [24]:
S = [simulation_dictionary[i] for i ∈ 1:number_of_simulation_steps]; # put the s_1, ...., s_n into a basket
NS₁ = findall(x-> x == 1, S) |> length; # how many 1's in the basket?
NS₂ = findall(x-> x == 2, S) |> length; # how many 2's in the basket?
NS₃ = findall(x-> x == 3, S) |> length; # how many 3's in the basket?

We can compute the relative frequency (probability) of the states `s = 1`, `s = 2`, and `s = 3`:

In [26]:
PS1 = NS₁/number_of_simulation_steps;
PS2 = NS₂/number_of_simulation_steps;
PS3 = NS₃/number_of_simulation_steps;
println("The sampled probability (p₁,p₂,p₃) = ($(PS1), $(PS2), $(PS3))")

The sampled probability (p₁,p₂,p₃) = (0.2718, 0.4352, 0.293)


### What does it mean when we sample $\bar{\pi}$ directly?
A common source of confusion is the meaning of the stationary distribution $\bar{\pi}$. 

Suppose we don't care about the system's dynamics (time behavior). Instead, we are only interested in the steady-state behavior, i.e., the stationary distribution. In that case, we can sample $\bar{\pi}$ directly, but if we do that, we lose all the information about the transitions, i.e., all the time behavior. To explore this, consider the following thought experiment:

* __Thought experiment__: Suppose our three-state model represented investor mood. We gathered `20` investors together in a focus group, isolated them, but gave them access to the same sources of information, and watched their mood for many rounds, i.e., as $n\rightarrow\infty$. The long-term mood of each investor would be samples from the stationary distribution $\bar{\pi}$

Let's implement this thought experiment. Create a [categorical distribution](https://en.wikipedia.org/wiki/Categorical_distribution) using the stationary probability of our Markov chain using the [Distributions.jl](https://github.com/JuliaStats/Distributions.jl) package, save this distribution in the variable `d`:

In [28]:
d = Categorical(π̄[1,:]); # build a model of the stationary distribution

We can then generate samples from the [categorical distribution](https://en.wikipedia.org/wiki/Categorical_distribution) saved in the distribution `d` using the [rand function](https://docs.julialang.org/en/v1/stdlib/Random/#Base.rand). This allows us to `simulate` the long-time behavior encoded by our three-state Markov model, i.e., the moods of the investors in our focus group.

In [30]:
rand(d,20) # generate 20 samples from the stationary distribution

20-element Vector{Int64}:
 2
 2
 2
 1
 2
 3
 2
 2
 3
 1
 1
 3
 2
 2
 2
 3
 1
 2
 2
 3

### Wait, why are there illegal transitions?
Sampling the stationary distribution $\bar{\pi}$ does __not give information about dynamics__. These samples do __not__ represent the transition of states between time steps. 

Instead, the stationary distribution $\bar{\pi}$ gives us the relative likelihood that we observed state $s_{i}\in\mathcal{S}$ as $n\rightarrow\infty$ (at long times). If we want information about the dynamics, we need to sample the chain directly __not__ the stationary distribution.